# Here I am trying to predict wheather the price of a crytocurrency will rise or fall depending on the prvious sequential data.

# Data link: https://drive.google.com/file/d/1gqd1It7miG_lrSbUrWfoEop9hKlV9tQ9/view?usp=sharing

# We are working with data of 4 different Crypto currencies.
### They are Bitcoin, Litecoin, Ethereum and Bitcoin cash.

# Importing necessary libraries and inititialising certain constants

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from collections import deque
import random
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,LSTM,BatchNormalization
#from tensorflow.keras.callbacks import Tensorboard, ModelCheckpoint

seq_len = 60 #length of sequence taken as input
future_period_predict = 3 #will predict 3 mins into the future
ratio_to_predict = "LTC-USD" # Crypto currency currently being predicted
EPOCHS = 10
BATCH = 64
#name = f"{seq_len}-seq-{future_period_predict}-pred-{int(time.time())}"

# Combining all the dataframe to make the desired DF

In [4]:
main_df = pd.DataFrame()

In [5]:
# combiing the dataframe
ratios = ["BTC-USD","LTC-USD","ETH-USD","BCH-USD"]
for ratio in ratios:
    dataset = f"{ratio}.csv"
    df = pd.read_csv(dataset,names=["time","low","high","open","close","volume"])
    #print(df)
    df.rename(columns = {"close":f"{ratio}_close","volume": f"{ratio}_volume"},inplace = True)
    df.set_index("time",inplace= True)
    df = df[[f"{ratio}_close",f"{ratio}_volume"]]
    #print(df.head())
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)
main_df

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,ETH-USD_close,ETH-USD_volume,BCH-USD_close,BCH-USD_volume
time,,,,,,,,
1528968660,6489.549805,0.587100,96.580002,9.647200,NaN,NaN,871.719971,5.675361
1528968720,6487.379883,7.706374,96.660004,314.387024,486.010010,26.019083,870.859985,26.856577
1528968780,6479.410156,3.088252,96.570000,77.129799,486.000000,8.449400,870.099976,1.124300
1528968840,6479.410156,1.404100,96.500000,7.216067,485.750000,26.994646,870.789978,1.749862
1528968900,6479.979980,0.753000,96.389999,524.539978,486.000000,77.355759,870.000000,1.680500
...,...,...,...,...,...,...,...,...
1535214960,6713.140137,0.769891,58.020000,6.434783,279.359985,11.280577,531.479980,1.208560
1535215020,6714.520020,1.002652,58.009998,7.301921,279.359985,8.790519,531.479980,0.016868
1535215080,6714.520020,1.021925,58.020000,23.802017,279.369995,1.311763,531.469971,0.013854


In [6]:
main_df.columns

Index(['BTC-USD_close', 'BTC-USD_volume', 'LTC-USD_close', 'LTC-USD_volume',
       'ETH-USD_close', 'ETH-USD_volume', 'BCH-USD_close', 'BCH-USD_volume'],
      dtype='object')

In [7]:
main_df.head()

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,ETH-USD_close,ETH-USD_volume,BCH-USD_close,BCH-USD_volume
time,,,,,,,,
1528968660,6489.549805,0.587100,96.580002,9.647200,NaN,NaN,871.719971,5.675361
1528968720,6487.379883,7.706374,96.660004,314.387024,486.01001,26.019083,870.859985,26.856577
1528968780,6479.410156,3.088252,96.570000,77.129799,486.00000,8.449400,870.099976,1.124300
1528968840,6479.410156,1.404100,96.500000,7.216067,485.75000,26.994646,870.789978,1.749862
1528968900,6479.979980,0.753000,96.389999,524.539978,486.00000,77.355759,870.000000,1.680500


# Creating the Training data

## Making data that the model needs to predict

In [8]:
def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

In [9]:
main_df["future"] = main_df[f"{ratio_to_predict}_close"].shift(-future_period_predict)

In [10]:
# Updated DF with the future value after 3 minutes
main_df.head()

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,ETH-USD_close,ETH-USD_volume,BCH-USD_close,BCH-USD_volume,future
time,,,,,,,,,
1528968660,6489.549805,0.587100,96.580002,9.647200,NaN,NaN,871.719971,5.675361,96.500000
1528968720,6487.379883,7.706374,96.660004,314.387024,486.01001,26.019083,870.859985,26.856577,96.389999
1528968780,6479.410156,3.088252,96.570000,77.129799,486.00000,8.449400,870.099976,1.124300,96.519997
1528968840,6479.410156,1.404100,96.500000,7.216067,485.75000,26.994646,870.789978,1.749862,96.440002
1528968900,6479.979980,0.753000,96.389999,524.539978,486.00000,77.355759,870.000000,1.680500,96.470001


### Creating the target label

##### Determine if the price goes up "1" or Down "0" by comparing current value and future value

In [11]:
main_df["target"] = list(map(classify,main_df[f"{ratio_to_predict}_close"],main_df["future"]))

In [12]:
# updated DF
main_df.head(10)

,BTC-USD_close,BTC-USD_volume,LTC-USD_close,LTC-USD_volume,ETH-USD_close,ETH-USD_volume,BCH-USD_close,BCH-USD_volume,future,target
time,,,,,,,,,,
1528968660,6489.549805,0.587100,96.580002,9.647200,NaN,NaN,871.719971,5.675361,96.500000,0
1528968720,6487.379883,7.706374,96.660004,314.387024,486.01001,26.019083,870.859985,26.856577,96.389999,0
1528968780,6479.410156,3.088252,96.570000,77.129799,486.00000,8.449400,870.099976,1.124300,96.519997,0
1528968840,6479.410156,1.404100,96.500000,7.216067,485.75000,26.994646,870.789978,1.749862,96.440002,0
1528968900,6479.979980,0.753000,96.389999,524.539978,486.00000,77.355759,870.000000,1.680500,96.470001,1
1528968960,6480.000000,1.490900,96.519997,16.991997,486.00000,7.503300,869.989990,1.669014,96.400002,0
1528969020,6477.220215,2.731950,96.440002,95.524078,485.98999,85.877251,869.450012,0.865200,96.400002,0
1528969080,6480.000000,2.174240,96.470001,175.205307,485.98999,160.915192,869.989990,23.534929,96.400002,0
1528969140,6479.990234,0.903100,96.400002,43.652802,485.98999,61.371887,870.000000,2.300000,96.400002,0


# selecting out of sample data

### Spliting the data into training and testing

In [13]:
times = sorted(main_df.index.values)
last_5pct = times[-int(0.05*len(times))]
print(last_5pct)#time stamp
# split is being done based on this time stamps

1534922100


### data with timestamp higher than last_5pct becomes the validation data

In [14]:
validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]

# Preprocessing the data

## The function created does the following tasks
1. Drops the future column which is no longer needed.
1. Normalise and scales all the column data into the same range.
1. Creates sequntial data as a list of lists, where 1st element of the list is the previous 60 mins of data and 2nd element if the label.
1. Balances the data such that the data points in the 2 classes are of the same count.
1. separates the sequential data into 2 varibles ; x store the sequnces and y stores the labels.

In [15]:
# function to preprocess
def preprocess_df(df):
    df = df.drop("future",1)
    for col in df.columns:
        if col != "target":
            df[col] = df[col].pct_change()
            df.dropna(inplace =True)
            df[col] = preprocessing.scale(df[col].values)
            
    df.dropna(inplace = True)
    
    sequential_data = []
    prev_days = deque(maxlen = seq_len)
    
    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        if len(prev_days) == seq_len:
            sequential_data.append([np.array(prev_days),i[-1]])
    random.shuffle(sequential_data)
    # BALANCING
    buys = []
    sells =[]
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq,target])
        elif target == 1:
            buys.append([seq,target])
            
    random.shuffle(buys)
    random.shuffle(sells)
    
    lower = min(len(buys),len(sells))

    buys = buys[:lower]
    sells = sells[:lower]
            
    sequential_data = buys + sells
    random.shuffle(sequential_data)
    
    x = []
    y = []
    for seq, target in sequential_data:
        x.append(seq)
        y.append(target)
        
    return np.array(x),np.array(y) 

# Creating training and testing data.

In [16]:
train_df, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

In [17]:
print(f"train data: {len(train_df)} validation: { len(validation_x)}")

train data: 69188 validation: 3062


In [18]:
train_df.shape

(69188, 60, 8)

In [19]:
len(train_y)

69188

In [20]:
train_y[1]

1.0

In [21]:
#train_df[1]

# model buliding

In [22]:
model = Sequential()
model.add(LSTM(128,input_shape = (train_df.shape[1:]),return_sequences = True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128,input_shape = (train_df.shape[1:]),return_sequences = True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128,input_shape = (train_df.shape[1:])))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32,activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(2,activation = "softmax"))

opt  = tf.keras.optimizers.Adam(lr=0.001,decay = 1e-6 )
model.compile(loss = "sparse_categorical_crossentropy",
             optimizer = opt,
             metrics = ["accuracy"])


In [23]:
model.fit(train_df,train_y,batch_size = BATCH,epochs=EPOCHS,validation_data=(validation_x,validation_y))

Epoch 1/10
1082/1082 [==============================] - 200s 179ms/step - loss: 0.7465 - accuracy: 0.5184 - val_loss: 0.6870 - val_accuracy: 0.5480
Epoch 2/10
1082/1082 [==============================] - 187s 173ms/step - loss: 0.6885 - accuracy: 0.5436 - val_loss: 0.6811 - val_accuracy: 0.5728
Epoch 3/10
1082/1082 [==============================] - 199s 184ms/step - loss: 0.6829 - accuracy: 0.5594 - val_loss: 0.6839 - val_accuracy: 0.5500
Epoch 4/10
1082/1082 [==============================] - 194s 180ms/step - loss: 0.6813 - accuracy: 0.5615 - val_loss: 0.6770 - val_accuracy: 0.5585
Epoch 5/10
1082/1082 [==============================] - 195s 180ms/step - loss: 0.6805 - accuracy: 0.5694 - val_loss: 0.6778 - val_accuracy: 0.5705
Epoch 6/10
1082/1082 [==============================] - 207s 191ms/step - loss: 0.6768 - accuracy: 0.5749 - val_loss: 0.6807 - val_accuracy: 0.5601
Epoch 7/10
1082/1082 [==============================] - 210s 194ms/step - loss: 0.6755 - accuracy: 0.5760 - val_

In [24]:
model.save("crypto.h5")

# When it comes to predicting finances, an accuracy of around 60% is considered very impressive. Here we can see that our model has achieved an acuuracy of about 57%

# The accuracy can be further improved with more number of epochs and some hyperparameter tuning.

# Changing the value of 'ratio_to_predict', we can create a model for other Cryptocurrencies as well.